## Installing all the necessary librarires and loading data from gcp

In [ ]:
import pandas as pd
from google.cloud import storage
import os
%pip install fuzzywuzzy
from google.cloud import bigquery


In [42]:
credentials_path = os.path.join("..", os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
client = storage.Client.from_service_account_json(json_credentials_path=credentials_path)

In [ ]:
bucket_name = os.getenv('BUCKET_NAME')
colonias_geo = 'georef-mexico-colonia.geojson'
colonias_csv = 'georef-mexico-colonia.csv'

bucket = client.get_bucket(bucket_name)
blob_geo = bucket.blob(colonias_geo)
blob_csv = bucket.blob(colonias_csv)


blob_geo.download_to_filename('local_geo.json')
blob_csv.download_to_filename('local_csv.csv')

In [ ]:
# Read the downloaded GeoJSON file
geojson_file = 'local_geo.json'
df_geojson = pd.read_json(geojson_file)

# Read the downloaded CSV file
csv_file = 'local_csv.csv'
df_csv = pd.read_csv(csv_file, delimiter=";")

# Actual preprocessing starts here 

In [ ]:
#download the colonia_names file (2041 colonias)
url = 'https://storage.googleapis.com/safety-map-model/coords_colonias.csv'
colonias_names = pd.read_csv(url)

In [ ]:
#cleaning the colonia names to compare them
colonias_names['cleaned_names'] = colonias_names['alcaldia_colonia'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')


In [ ]:
merged_names = df_geojson['features'].apply(lambda x: f"{x['properties']['mun_name'][0]} - {x['properties']['col_name'][0]}")
df_geojson['merged_names'] = merged_names

In [ ]:
df_geojson['geometry_data'] = df_geojson['features'].apply(lambda x: x['geometry'])
polygon_colonias = df_geojson[['merged_names', 'geometry_data']]

In [ ]:
polygon_colonias['cleaned_names'] = polygon_colonias['merged_names'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')

In [ ]:
# Dictionary mapping Spanish characters to English letters
spanish_to_english = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'ñ': 'n',
}

# Replace Spanish characters with English letters
polygon_colonias['cleaned_names'] = polygon_colonias['cleaned_names'] .str.replace('|'.join(spanish_to_english.keys()), lambda x: spanish_to_english[x.group()])


In [ ]:
merged_df_for_lists = colonias_names.merge(polygon_colonias, on='cleaned_names', how='outer')


In [ ]:
nan_values_colonia_names = merged_df_for_lists[merged_df_for_lists['merged_names'].isna()]['cleaned_names'].tolist()
nan_values_polygons = merged_df_for_lists[merged_df_for_lists['alcaldia_colonia'].isna()]['cleaned_names'].tolist()


# Merging the df_csv (including the polygons) with the colonia names df 

In [ ]:
#Clean the df_csv file 
df_csv['merged_names'] = df_csv['Municipality name'] + "-" + df_csv['Colonia name']
df_csv['cleaned_names'] = df_csv['merged_names'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')


In [ ]:
df_csv['code'] = df_csv['Geo Point'].apply(lambda x: x.split(',')[0].strip()).astype(float)
df_csv['numeric'] = list(range(len(df_csv)))

In [ ]:
# Dictionary mapping Spanish characters to English letters
spanish_to_english = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'ñ': 'n',
}

# Replace Spanish characters with English letters
df_csv['cleaned_names'] = df_csv['cleaned_names'] .str.replace('|'.join(spanish_to_english.keys()), lambda x: spanish_to_english[x.group()])


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# creating the dictionaries
dict1 = {'name': nan_values_polygons}
dict2 = {'name': nan_values_colonia_names}




# converting to pandas dataframes
dframe1 = pd.DataFrame(dict1)
dframe2 = pd.DataFrame(dict2)

# empty lists for storing the matches later
mat1 = []

# converting to pandas dataframes
dframe1 = pd.DataFrame(dict1)
dframe2 = pd.DataFrame(dict2)
  
# empty lists for storing the matches later
mat1 = []
mat2 = []
p = []
  
# printing the pandas dataframes
print("First dataframe:\n", dframe1, 
      "\nSecond dataframe:\n", dframe2)
  
# converting dataframe column to list
# of elements
# to do fuzzy matching
list1 = dframe1['name'].dropna().tolist()
list2 = dframe2['name'].dropna().tolist()
  
# taking the threshold as 82
threshold = 65
  
# iterating through list1 to extract 
# it's closest match from list2
for i in list1:
    mat1.append(process.extract(i, list2, limit=2, scorer=fuzz.partial_ratio))
dframe1['matches'] = mat1
  
# iterating through the closest matches
# to filter out the maximum closest match
for i in list1:
    match = process.extractOne(i, list2, scorer=fuzz.partial_ratio)
    if match[1] >= threshold:
        mat2.append(match[0])
    else:
        mat2.append(None)
  
  
# storing the resultant matches back to dframe1
dframe1['matches'] = mat2
print("\nDataFrame after Fuzzy matching:")
dframe1


In [ ]:
fuzzy_dict_new = dframe1.set_index("name")['matches'].to_dict()

In [ ]:
fuzzy_dict_new

In [ ]:
"xochimilcoloscerrillosi" in df_csv.values

In [ ]:
df_csv['cleaned_names'] = df_csv['cleaned_names'].replace(fuzzy_dict_new)

In [ ]:
final_merge_new = colonias_names.merge(df_csv, on="cleaned_names", how= "inner")

In [ ]:
upload_file_polygons = final_merge_new[['alcaldia_colonia', 'latitud', 'longitud', 'code', 'Municipality name', 'Colonia name', 'cleaned_names', 'Colonia code']]
upload_file_polygons.rename(columns= lambda x: x.lower().replace(" ", "_"), inplace=True)
upload_file_polygons.rename(columns={'alcaldia_colonia': 'Neighborhood'}, inplace= True)

In [43]:
upload_file_polygons

,Neighborhood,latitud,longitud,code,municipality_name,colonia_name,cleaned_names,colonia_code
0,ALVARO OBREGON || 19 DE MAYO,19.361684,-99.253435,19.361901,Álvaro Obregón,19 De Mayo,alvaroobregon19demayo,10-241
1,ALVARO OBREGON || 1RA VICTORIA,19.386773,-99.201722,19.386825,Álvaro Obregón,1ra Victoria,alvaroobregon1ravictoria,10-242
2,ALVARO OBREGON || 1RA VICTORIA SECCION BOSQUES,19.387465,-99.197754,19.387225,Álvaro Obregón,1ra Victoria Seccion Bosques,alvaroobregon1ravictoriaseccionbosques,10-243
3,ALVARO OBREGON || 26 DE JULIO,19.357406,-99.250817,19.357519,Álvaro Obregón,26 De Julio,alvaroobregon26dejulio,10-244
4,ALVARO OBREGON || 2DA JALALPA TEPITO (AMPL),19.374833,-99.234288,19.375635,Álvaro Obregón,2da Jalalpa Tepito (Ampl),alvaroobregon2dajalalpatepitoampl,10-246
...,...,...,...,...,...,...,...,...
1850,XOCHIMILCO || TIERRA NUEVA,19.266145,-99.119611,19.265977,Xochimilco,Tierra Nueva,xochimilcotierranueva,13-071
1851,XOCHIMILCO || TLAXOPAN,19.239229,-99.142730,19.233239,Xochimilco,Tlaxopan,xochimilcotlaxopan,13-072
1852,XOCHIMILCO || VILLA XOCHIMILCO (U HAB),19.267237,-99.126181,19.267183,Xochimilco,Villa Xochimilco (U Hab),xochimilcovillaxochimilco,13-083
1853,XOCHIMILCO || XALTOCAN (BARR),19.253543,-99.101498,19.252834,Xochimilco,Xaltocan (Barr),xochimilcoxaltocanbarr,13-073


In [ ]:
credentials_path = os.path.join("..", os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
client_bq = bigquery.Client.from_service_account_json(json_credentials_path=credentials_path)

In [44]:
#function to load csv into gbq
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool,
        client = None
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """

    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"

    # Load data onto full_table_name

    # 🎯 HINT for "*** TypeError: expected bytes, int found":
    # After preprocessing the data, your original column names are gone (print it to check),
    # so ensure that your column names are *strings* that start with either
    # a *letter* or an *underscore*, as BQ does not accept anything else

    # TODO: simplify this solution if possible, but students may very well choose another way to do it
    # We don't test directly against their own BQ tables, but only the result of their query
    data.columns = [f"_{column}" if not str(column)[0].isalpha() and not str(column)[0] == "_" else str(column) for column in data.columns]


    # Define write mode and schema
    write_mode = "WRITE_TRUNCATE" if truncate else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    print(f"\n{'Write' if truncate else 'Append'} {full_table_name} ({data.shape[0]} rows)")

    # Load data
    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)
    result = job.result()  # wait for the job to complete

    print(f"✅ Data saved to bigquery, with shape {data.shape}")

In [45]:
load_data_to_bq(data = upload_file_polygons, gcp_project = os.getenv("GCP_PROJECT"), bq_dataset =os.getenv("BQ_DATASET"),   table = "neighborhood_polygon", truncate = True, client = client_bq )


Write wagon-bootcamp-385417.Safetymap.neighborhood_polygon (1855 rows)
✅ Data saved to bigquery, with shape (1855, 8)


In [ ]:
import folium

In [48]:
upload_file_polygons

,Neighborhood,latitud,longitud,code,municipality_name,colonia_name,cleaned_names,colonia_code
0,ALVARO OBREGON || 19 DE MAYO,19.361684,-99.253435,19.361901,Álvaro Obregón,19 De Mayo,alvaroobregon19demayo,10-241
1,ALVARO OBREGON || 1RA VICTORIA,19.386773,-99.201722,19.386825,Álvaro Obregón,1ra Victoria,alvaroobregon1ravictoria,10-242
2,ALVARO OBREGON || 1RA VICTORIA SECCION BOSQUES,19.387465,-99.197754,19.387225,Álvaro Obregón,1ra Victoria Seccion Bosques,alvaroobregon1ravictoriaseccionbosques,10-243
3,ALVARO OBREGON || 26 DE JULIO,19.357406,-99.250817,19.357519,Álvaro Obregón,26 De Julio,alvaroobregon26dejulio,10-244
4,ALVARO OBREGON || 2DA JALALPA TEPITO (AMPL),19.374833,-99.234288,19.375635,Álvaro Obregón,2da Jalalpa Tepito (Ampl),alvaroobregon2dajalalpatepitoampl,10-246
...,...,...,...,...,...,...,...,...
1850,XOCHIMILCO || TIERRA NUEVA,19.266145,-99.119611,19.265977,Xochimilco,Tierra Nueva,xochimilcotierranueva,13-071
1851,XOCHIMILCO || TLAXOPAN,19.239229,-99.142730,19.233239,Xochimilco,Tlaxopan,xochimilcotlaxopan,13-072
1852,XOCHIMILCO || VILLA XOCHIMILCO (U HAB),19.267237,-99.126181,19.267183,Xochimilco,Villa Xochimilco (U Hab),xochimilcovillaxochimilco,13-083
1853,XOCHIMILCO || XALTOCAN (BARR),19.253543,-99.101498,19.252834,Xochimilco,Xaltocan (Barr),xochimilcoxaltocanbarr,13-073


In [49]:
#test if everything worked and build a map
state_geo = 'local_geo.json'


map = folium.Map(location=[19.4326, -99.1332], zoom_start=11, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=upload_file_polygons,
    columns=["code", "score"],
    key_on="feature.properties.geo_point_2d.lat",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(map)

folium.LayerControl().add_to(map)

map

KeyError: 'score'